#Basic info!!!


I tried a lot of models like LSTM, Deep NN, ARIMA, Catboost, LGBoost, etc etc.

Surprised to say among all those, I got the best results (atleast for my implementations) for XGboost which is one of the first models I tried actually

Required libraries :
Pandas, Scikit-Learn,XGBoost

DM me on whatsapp if youd like to see the other barely working models or would like explanation anywhere


Place test.csv and train.csv in the location of this code before running!!!!


In [67]:
#Train Data pre-processing
import pandas as pd
df = pd.read_csv('./train.csv')
df.head()


,id,Date,Rain,Temp Max,Temp Min
0,0,01-01-1951,0.0,28.940001,15.58
1,1,01-02-1951,0.0,27.889999,13.86
2,2,01-03-1951,0.0,27.320000,12.38
3,3,01-04-1951,0.0,27.670000,13.06
4,4,01-05-1951,0.0,28.320000,14.79


In [68]:
dateList = pd.date_range(start='1951-01-01', end='2009-12-31').tolist()
df['Date'] = dateList

dates = []
months = []
years = []
for x in df['Date'] :
    temp = str(x).split("-")
    years.append(temp[0])
    months.append(temp[1])
    dates.append(temp[2][:2])
df['Full date'] = df['Date']
df['Year'] = years 
df['Month'] = months
df['Date'] = dates
df.to_csv('./processedData.csv', index=False) #saving in a file for later usage
df.head()

,id,Date,Rain,Temp Max,Temp Min,Full date,Year,Month
0,0,01,0.0,28.940001,15.58,1951-01-01,1951,01
1,1,02,0.0,27.889999,13.86,1951-01-02,1951,01
2,2,03,0.0,27.320000,12.38,1951-01-03,1951,01
3,3,04,0.0,27.670000,13.06,1951-01-04,1951,01
4,4,05,0.0,28.320000,14.79,1951-01-05,1951,01


In [69]:
#Prediction data preprocessing 

# Data Preprocessing
df = pd.read_csv('./test.csv')
dateList = pd.date_range(start='2010-01-01', end='2020-10-24').tolist()
df['Date'] = dateList
df.head()

,id,Date
0,21550,2010-01-01
1,21551,2010-01-02
2,21552,2010-01-03
3,21553,2010-01-04
4,21554,2010-01-05


In [70]:
dates = []
months = []
years = []
for x in df['Date'] :
    temp = str(x).split("-")
    years.append(temp[0])
    months.append(temp[1])
    dates.append(temp[2][:2])
df['Full date'] = df['Date']
df['Year'] = years 
df['Month'] = months
df['Date'] = dates

df.to_csv('./processedTest.csv', index=False) #make prototypes in knime
df.head()

,id,Date,Full date,Year,Month
0,21550,01,2010-01-01,2010,01
1,21551,02,2010-01-02,2010,01
2,21552,03,2010-01-03,2010,01
3,21553,04,2010-01-04,2010,01
4,21554,05,2010-01-05,2010,01


In [71]:
#Train model for max temperature using XGBooster
df = pd.read_csv('./processedData.csv')
from sklearn.preprocessing import MinMaxScaler

X = df[['Month', 'Date', 'Year']]
y = df['Temp Max']


#Applying normalization is decreasing accuracy


# #Normalization

# scaler_X = MinMaxScaler()
# scaler_y = MinMaxScaler()

# X_normalized = scaler_X.fit_transform(X)
# y_normalized = scaler_y.fit_transform(y.values.reshape(-1, 1)) #reshape to 2d array (needed for scaler if I remember)

In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
#Create XGBooster model for max temp 
import xgboost as xgb
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=10000, learning_rate=0.1)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [74]:
#Check r^2 on test set
from sklearn.metrics import r2_score, mean_absolute_error
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('R² Score:',r2)
print('Mean Absolute Error:',mae)

R² Score: 0.9036397916504184
Mean Absolute Error: 0.9312865865956056


In [75]:
#Apply predictions for max temp on processed test file
df = pd.read_csv('./processedTest.csv')
X = df[['Month', 'Date', 'Year']]
y_pred = model.predict(X)

df['maxTemp'] = y_pred

dfMaxTemp = df

In [76]:
#Now do the process for min temp
df = pd.read_csv('./processedData.csv')
X = df[['Month', 'Date', 'Year']]
y = df['Temp Min']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train,X_test,y_train,y_test
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=10000, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('R² Score:',r2)
print('Mean Absolute Error:',mae)

df = pd.read_csv('./processedTest.csv')
X = df[['Month', 'Date', 'Year']]
y_pred = model.predict(X)

df['minTemp'] = y_pred

df.to_csv("./minTempOutput.csv")

dfMinTemp = df
dfMinTemp

R² Score: 0.9324710820650346
Mean Absolute Error: 0.7568410094422431


,id,Date,Full date,Year,Month,minTemp
0,21550,1,2010-01-01,2010,1,16.818951
1,21551,2,2010-01-02,2010,1,15.868185
2,21552,3,2010-01-03,2010,1,15.840927
3,21553,4,2010-01-04,2010,1,15.135238
4,21554,5,2010-01-05,2010,1,16.004232
...,...,...,...,...,...,...
3945,25495,20,2020-10-20,2020,10,20.249073
3946,25496,21,2020-10-21,2020,10,19.541550
3947,25497,22,2020-10-22,2020,10,19.673658
3948,25498,23,2020-10-23,2020,10,18.541941


In [77]:
#now finally let us combine dfMaxTemp and dfMinTemp

combined_df = pd.merge(dfMaxTemp, dfMinTemp, on='id', how='outer')
filtered_df = combined_df[['Full date_x', 'maxTemp', 'minTemp']] #Dropping other columns
filtered_df.rename(columns={'Full date_x': 'Date'}, inplace=True)

filtered_df.to_csv("./Final output.csv")

C:\Users\manug\AppData\Local\Temp\ipykernel_19476\131817891.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'Full date_x': 'Date'}, inplace=True)
